In [3]:
import pandas as pd
import time
import os
import re
from sqlalchemy import create_engine
# Librerías para web scraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [2]:
pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.1 MB 1.9 MB/s eta 0:00:02
   -- ------------------------------------- 0.2/2.1 MB 1.8 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/2.1 MB 2.1 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/2.1 MB 2.1 MB/s eta 0:00:01
   --------- ------------------------------ 0.5/2.1 MB 2.0 MB/s eta 0:00:01
   ---------- ----------------------------- 0.5/2.1 MB 1.9 MB/s eta 0:00:01
   ------------ --------------------------- 0.6/2.1 MB 1.9 MB/s eta 0:00:01
   -------------- ------------------------- 0.7/2.1 MB 1.8 MB/s eta 0:00:01
   ---------------- ----------------------- 0.9/2.1 MB 2.0 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 2.1 MB/s eta 0:00:01
   --------------------- ------------------ 1.1/2.1 MB 2.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.2 MB/s eta 0:00:01
   ----------------

In [4]:
# Configurar opciones de visualización
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.max_colwidth', None) # Mostrar todo el valor de la columna

In [17]:
driver_path = r"D:\Drivers\chromedriver.exe"
web = "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx.html"

In [18]:
# Calibrar opciones y el service necesario
options = Options()
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
service = Service(driver_path)

In [19]:
# Acceso a la web de Volvo
url = webdriver.Chrome(service=service, options=options)
url.get(web)

In [20]:
# Encuentra todos los enlaces de productos
productos = url.find_elements(By.XPATH, '//*[@id="layer-product-list"]/div[2]/ol/li/div/div[1]/a')

In [21]:
event_labels = []
ids = []
prices = []

In [22]:
def scrap():
    while True:
        # Encuentra todos los enlaces de productos en la página actual
        productos = url.find_elements(By.XPATH, '//*[@id="layer-product-list"]/div[2]/ol/li/div/div[1]/a')
        
        for producto in productos:
            try:
                # Extraer los datos del atributo 'onclick' usando JavaScript
                onclick_text = producto.get_attribute('onclick')
                
                # Extraer eventLabel
                event_label = onclick_text.split('"eventLabel":"')[1].split('",')[0]
                event_labels.append(event_label)
                
                # Extraer id
                id_value = onclick_text.split('"id":"')[1].split('",')[0]
                ids.append(id_value)

                # Extraer price
                price = onclick_text.split('"price":"')[1].split('",')[0]
                prices.append(price)

            except Exception as e:
                # En caso de que algún campo no se pueda extraer, agregar un valor nulo o mensaje de error
                event_labels.append('N/A')
                ids.append('N/A')
                prices.append('N/A')
                print(f"Error al extraer datos del producto: {e}")
        
        return event_labels, ids, prices

In [23]:
def next_page():
    # Intentar encontrar el botón de "Siguiente" y hacer clic en él
    try:
        next_button = url.find_element(By.XPATH, '//*[@id="layer-product-list"]/div[3]/div[3]/ul/li/a[@class="action  next"]')
        next_button.click()
        time.sleep(0.5)  # Esperar un momento para que la nueva página cargue completamente
        
    except Exception as e:
        return print("Extracción terminada")  # Indicar que no se pudo avanzar a la siguiente página

In [24]:
%%time
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
event_labels, ids, prices = scrap() # Scrapear página
next_page() # pasar a la siguiente página
url.quit()

Extracción terminada
CPU times: total: 15.6 ms
Wall time: 39.4 s


In [25]:
# Crear un DataFrame con los datos extraídos
df = pd.DataFrame({
    'Producto': event_labels,
    'ID': ids,
    'Precio': prices
})

In [26]:
df

,Producto,ID,Precio
0,Barra de reacci\u00f3n,VO-21051046,1150.05
1,Cojinete de rodillo,VO-20582549,482.14
2,Panel Interruptor,VO-22566510,249.28
3,Anillo de embrague,VO-1668452,853.87
4,Arandela de ajuste,VO-25640755,63.61
5,Anillo,VO-943760,14.03
6,"Faro posterior combinado, derecho",VO-23522308,667.88
7,"Faro posterior combinado, izquierdo",VO-23522307,667.88
8,conector,VO-1547254,7.61
9,conector,VO-21021925,38.18


In [14]:
# Buscamos que records contienen caracteres especiales
df[df['Producto'].str.contains(r'\\')]

,Producto,ID,Precio
0,Barra de reacci\u00f3n,VO-21051046,1150.05
11,Juego de aro de pist\u00f3n,VO-20509932,168.07
12,Anillo ret\u00e9n,VO-21503575,6.48
33,Kit Tubos de Direcci\u00f3n,VO-23942786,295.58
43,Sensor de Veloc\u00edmetro,VO-24162657,231.83
47,Interruptor de presi\u00f3n,VO-20424051,165.64
50,Moldura de ret\u00e9n,VO-20479636,20.06
51,Moldura de ret\u00e9n,VO-20489341,39.77
52,Moldura de ret\u00e9n,VO-20714247,36.04
68,Balanc\u00edn de inyector,VO-21881195,513.26


In [27]:
def limpiar_caracteres_especiales(text):
    # Reemplazar caracteres especiales como '\u00f3' con su equivalente legible
    text = re.sub(r'\\u00([0-9a-fA-F]{2})', lambda m: chr(int(m.group(1), 16)), text)
    # Reemplazar slash con espacio
    text = text.replace('/', ' ')
    return text

In [28]:
# Aplicar la función limpiar_caracteres_especiales a la columna "Producto"
df['Producto'] = df['Producto'].apply(limpiar_caracteres_especiales)
# Convertir la columna "Precio" de string a float
df['Precio'] = df['Precio'].astype(float)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Producto  308 non-null    object 
 1   ID        308 non-null    object 
 2   Precio    308 non-null    float64
dtypes: float64(1), object(2)
memory usage: 7.3+ KB


In [30]:
df

,Producto,ID,Precio
0,Barra de reacción,VO-21051046,1150.05
1,Cojinete de rodillo,VO-20582549,482.14
2,Panel Interruptor,VO-22566510,249.28
3,Anillo de embrague,VO-1668452,853.87
4,Arandela de ajuste,VO-25640755,63.61
5,Anillo,VO-943760,14.03
6,"Faro posterior combinado, derecho",VO-23522308,667.88
7,"Faro posterior combinado, izquierdo",VO-23522307,667.88
8,conector,VO-1547254,7.61
9,conector,VO-21021925,38.18


In [54]:
%%time
# CARGA A SQL
try:
    # Crear la conexión a SQL Server con SQLAlchemy
    connection_string = (
        "mssql+pyodbc://DESKTOP-O6LKN6I/"  # En vez de "localhost" cambiarlo por tu servidor
        "oficina?" # En vez de "TSQL" cambiarlo por tu Base de Datos
        "driver=ODBC+Driver+17+for+SQL+Server"
    )
    # Crear la conexión a SQL Server con SQLAlchemy
    engine = create_engine(connection_string, echo=True)
    # Nombre de la tabla y esquema en SQL Server
    nombre_tabla = 'VOLVO'
    # Cargar los datos en la tabla de SQL Server
    print(df.to_sql(name=nombre_tabla, con=engine, index=False, if_exists='replace'))
    print(f'Data cargada exitosamente en SQL en la tabla: "{nombre_tabla}"')
except Exception as e:
    print(f'Error: {e}')
finally:
    # No es necesario cerrar la conexión en este caso ya que SQLAlchemy maneja la conexión automáticamente
    pass

2024-05-27 12:28:06,377 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-05-27 12:28:06,385 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-27 12:28:06,392 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-05-27 12:28:06,392 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()
2024-05-27 12:28:06,431 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-05-27 12:28:06,432 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ()
2024-05-27 12:28:06,436 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2024-05-27 12:28:06,437 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ()
2024-05-27 12:28:06,624 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-27 12:28:06,631 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLE

In [53]:
pip install pyodbc


   ---------------------------------------- 0.0/69.3 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/69.3 kB ? eta -:--:--
   ---------------------------------------- 69.3/69.3 kB 1.3 MB/s eta 0:00:00
